# Código eficiente

## 1. Benchmarking

Usaremos las siguientes herramientas:
- @btime
- @benchmark
- BenchmarkTools.DEFAULT_PAREMETERS.samples
- BenchmarkTools.DEFAULT_PARAMETERS.seconds

In [ ]:
using BenchmarkTools
using LinearAlgebra

In [ ]:
A = rand(1000,1000)
@btime norm(A)

In [ ]:
function tiempo_funcion()
    B = rand(1000, 1000)
    @btime norm($B)
end

tiempo_funcion()

In [ ]:
A = rand(10_000, 10_000)
b = @benchmark norm(A)

In [ ]:
b.times

In [ ]:
b.times.*10^-9

In [ ]:
b = @benchmark(norm($A), samples=10)

In [ ]:
BenchmarkTools.DEFAULT_PARAMETERS.samples = 50

In [ ]:
b = @benchmark norm($A)

In [ ]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 50

In [ ]:
b = @benchmark norm($A)

## 2. Evitar tipos de datos abstractos 

Usemos las funciones `supertype()` y `subtype()`

In [ ]:
function registro_juegos_ganados(ngames)
    games_won = []   # inicializa un arreglo
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won, i)
        end
    end
    return games_won
end

ntrials = 1000
@btime registro_juegos_ganados(2)
@btime registro_juegos_ganados(ntrials)

In [ ]:
function registro_juegos_ganados_v2(ngames)
    games_won = Int64[]   # inicializa un arreglo de tipo Int64
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won, i)
        end
    end
    return games_won
end

ntrials = 1000
@btime registro_juegos_ganados_v2(2)
@btime registro_juegos_ganados_v2(ntrials);

In [ ]:
h = []
g = Int64[]

In [ ]:
@show typeof(h)

In [ ]:
@show typeof(g)

## 3. Evitar variables de ámbito global

In [ ]:
ntrials = 1000
allgames = rand(ntrials)

function registro_juegos_ganados_vglobal()
    games_won = Int64[]
    for (curi, curgame) in enumerate(allgames)
        if curgame >= 0.5
            push!(games_won, curi)
        end
    end
    return games_won
end


registro_juegos_ganados_vglobal();
registro_juegos_ganados_vglobal();

In [ ]:
@btime registro_juegos_ganados_vglobal();

In [ ]:
function registro_juegos_ganados_vlocal(ntrials)
    allgames = rand(ntrials)
    games_won = Int64[]
    for (curi, curgame) in enumerate(allgames)
        if curgame >= 0.5
            push!(games_won, curi)
        end
    end
    return games_won
end


registro_juegos_ganados_vlocal(ntrials);
registro_juegos_ganados_vlocal(ntrials);

In [ ]:
@btime registro_juegos_ganados_vlocal(ntrials);

Si es indispensable utilizar variables en el ámbito global, se decrararán con `const`. Aquí `const` no significa que el valor de la variable declarada no cambia, si no se refiere a que el tipo de la variable permancerá constante.  

In [ ]:
const myglobalint = 1
myglobalint = 2
@show myglobalint

In [ ]:
myglobalint = 1.5

## 4. Prealojamiento de memoria

En lugar de usar un vector de enteros que se inicializa vacío y añadirle elementos con la función `push!()`, es mejor crear un vector con una dimensión específica y se va agregando elemento a elemento en una posición determinada.

In [ ]:
function record_games_won_v2(ngames)
    games_won = Int64[]
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end

ntrials = 1000
record_games_won_v2(ntrials);

In [ ]:
@btime record_games_won_v2(ntrials);

In [ ]:
function record_games_won_preallocate(ntrials)
    allgames = rand(ntrials)
    games_won = Vector{Int64}(undef,ntrials)
    gi = 1
    for (curi,curgame) in enumerate(allgames)
        if curgame >= 0.5
            games_won[gi] = curi
            gi += 1
        end
    end
    return games_won[1:gi-1]
end

ntrials = 1000
record_games_won_preallocate(ntrials);

In [ ]:
@btime record_games_won_preallocate(ntrials);

## 5. Usar operaciones vectorizadas (Broadcasting)

In [ ]:
function record_games_won_preallocate(ntrials) 
    allgames = rand(ntrials)
    games_won = findall(allgames.>= 0.5)
    return games_won
end

ntrials = 1000;

In [ ]:
record_games_won_preallocate(ntrials); 
@btime record_games_won_preallocate(ntrials); 

In [ ]:
?findall

## 6. No es necesario vectorizar siempre las funciones

In [ ]:
function find_hypotenuse_vectorized(b,hi)
    return sqrt.(b.^2 + hi.^2)
end

b = rand(ntrials)
hi = rand(ntrials);

In [ ]:
@btime find_hypotenuse_vectorized($b,$hi);  # usa la funcion vectorizada
@btime find_hypotenuse_vectorized.($b,$hi);  # vectoriza la llamada

In [ ]:
function find_hypotenuse_forloop(b,hi)
    accum_vec = similar(b)  # preallocating memory
    for i = 1:length(b)
        accum_vec[i] = sqrt(b[i]^2 + hi[i]^2)
    end
    return accum_vec
end

b = rand(ntrials)
hi = rand(ntrials);

In [ ]:
@btime find_hypotenuse_forloop($b,$hi);  # usa una funcion convencional con prealojamiento de memoria

Se recomienda utilizar la operación de _broadcasting_ vectorizando la llamada a las funciones que reciben como argumentos vectores.

## 7. Reutilizar la memoria

In [ ]:
function find_sum_of_sqrt_vectors(nvectors)
    sumvector = Vector{Float64}(undef, nvectors)
    v = sqrt.(1:nvectors)
    for i = 1:nvectors
        sumvector[i] = sum(v[1:i])
    end
    return sumvector
end

ntrials = 1000
find_sum_of_sqrt_vectors(ntrials);

In [ ]:
@btime find_sum_of_sqrt_vectors(ntrials);

In [ ]:
function find_sum_of_sqrt_vectors_reusemem(nvectors)
    sumvector = Vector{Float64}(undef, nvectors)
    v = Vector{Float64}(undef, nvectors)
    for i = 1:nvectors
        v[1:i] .= sqrt.(1:i)
        sumvector[i] = sum(v)
        v .= 0
    end
    return sumvector
end

ntrials = 1000
find_sum_of_sqrt_vectors_reusemem(ntrials);

In [ ]:
@btime find_sum_of_sqrt_vectors_reusemem(ntrials);

En la segunda función se reutiliza la memoria apartada (_preallocated_) en el vector `v` por lo que no se hace ningún solicitud de memoria extra.

## 8. Use `view` cuando no se necesite una copia de datos

In [ ]:
using SparseArrays
using LinearAlgebra
using Random

In [ ]:
A = sprand(500, 500, 0.1)

function set_sum(A, rowids, colids)
    S = sum(A[rowids, colids])
end

set_sum(A, randperm(100), randperm(100));


In [ ]:
@btime set_sum($A, randperm(100), randperm(100));

In [ ]:
function set_sum_view(A, rowids, colids)
    S = sum(view(A, rowids, colids))
end

set_sum_view(A, randperm(100), randperm(100));

In [ ]:
@btime set_sum_view($A, randperm(100), randperm(100));

In [ ]:
function find_sum_of_sqrt_vectors_copies(nvectors)
    sumvector =  Vector{Float64}(undef, nvectors)
    v = sqrt.(1:nvectors)
    for i = 1:nvectors
        sumvector[i] = sum(v[1:i])
    end
    return sumvector
end

ntrials = 1000
@btime find_sum_of_sqrt_vectors_copies(ntrials);

In [ ]:
function find_sum_of_sqrt_vectors_views(nvectors)
    sumvector =  Vector{Float64}(undef, nvectors)
    v = sqrt.(1:nvectors)
    for i = 1:nvectors
        sumvector[i] = sum(@view v[1:i])
    end
    return sumvector
end

ntrials = 1000
@btime find_sum_of_sqrt_vectors_views(ntrials);


El subarreglo provisto por `@view` solo "observa" los datos pero no hace copia ni referencia a ellos.
***Nota:**


## 9. Revisa el código que estás generando

In [ ]:
f(x,y) = x * y

In [ ]:
f(3,4)

In [ ]:
f(3.4, 4.5)

In [ ]:
@code_lowered f(3,4)

In [ ]:
@code_typed f(3,4)

In [ ]:
@code_warntype f(3,4)

In [ ]:
@code_llvm f(3,4)

In [ ]:
@code_llvm f(3.5, 4)

In [ ]:
@code_native f(3.5, 4.5)

In [ ]:
@code_native f(3, 4.5)